In [1]:
from utils import PkdbModel,measurement_type_data, filter_out, convert_unit
from collections import namedtuple
import pandas as pd
from IPython.display import display, HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell    '''
    return HTML(
         '<table><tr style="background-color:white;">' +
         ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
         '</tr></table>'
     )

{'token': '55388d17d53f133a3f7eb7c69dd0874731fe3445'}


In [2]:
def codeine_n_idx(data):
    return (data.substance_intervention == 'codeine phosphate') \
           & (data.substance == 'codeine') \
           & (data['tissue'] == 'plasma')

In [3]:
def all_codeine_idx(data):
     return (data.substance_intervention == 'codeine phosphate') \
           |(data.substance_intervention == 'codeine sulphate') \
           |(data.substance_intervention == 'codeine') \
           |(data.substance == 'codeine') \
           & (data['tissue'] == 'plasma')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.to_numeric()
complete_codeine = all_complete.data[all_codeine_idx(all_complete.data)]
complete_codeine["inferred"] = False

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (25,28,29,30,31,44,72,88,95,114,115,117,119,121,123,124,126,127,128,129,132,133,134,136,140,142,146,155,157,158,159,161,162,168,169,170,171,180,181,182,183,185,186,188,190,192,197,198,199,200,207,208,212,215,216,224,229,233,234,239,243) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [6]:
Inference = namedtuple("Inference",["name","filter_out","measurement_type"])

In [7]:
inferences = [
    Inference(
        name="codeine_clearance",
        filter_out= {"unit":[ 'microgram * milliliter / hour / micromole']},
        measurement_type = "clearance",
    ),
    Inference(
        name="codeine_timecourse",
        filter_out= {"unit":["mole / liter"]},
        measurement_type = "concentration"
    ),
    Inference(
        name="codeine_thalf",
        filter_out= {},
        measurement_type = "thalf"
    ), 
    Inference(
        name="codeine_tmax",
        filter_out= {},
        measurement_type = "tmax"
    ),
    Inference(
        name="codeine_vd",
        filter_out= {"unit":[ 'liter * milligram / millimole']},
        measurement_type = "vd"
    ),
     Inference(
        name="codeine_auc_inf",
        filter_out= {},
        measurement_type = "auc_inf"
    ),
     Inference(
        name="codeine_auc_end",
        filter_out= {},
        measurement_type = "auc_end"
    ),
    Inference(
        name="codeine_kel",
        filter_out= {},
        measurement_type = "kel"
    ),
]

In [8]:
for inference in inferences:
    print(inference.name)
    data = measurement_type_data(complete_codeine,inference.measurement_type)
    data_class = PkdbModel(inference.name,destination="3-inferred")
    data_class.data = data
    [data_class.filter_out(k,v)for k,v in inference.filter_out.items()]
    data_class.infer_from_interventions()
    data_class.infer_from_outputs()
    data_class.save()

    t1 = pd.DataFrame(data_class.data.groupby(["unit", "unit_intervention"]).apply(len), columns=["count"])
    t2 = pd.DataFrame(data_class.data.groupby(["inferred"]).apply(len), columns=["count"])
    t3 = pd.DataFrame(data_class.data.groupby(["calculated"]).apply(len), columns=["count"])
    display(multi_table([t1,t2,t3]))

codeine_clearance


codeine_timecourse


codeine_thalf


codeine_tmax


codeine_vd


codeine_auc_inf


codeine_auc_end


codeine_kel
